In [ ]:
!pip install pypdf

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes  #bitsand bytes for quantizing into 4 bytes because llama uses 16 bytes

In [ ]:
#embedding
!pip install sentence_transformers

In [ ]:
!pip install llama_index

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext


In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM

In [ ]:
%pip install llama-index-llms-huggingface

In [ ]:
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [ ]:
documents=SimpleDirectoryReader("/content/data").load_data()

In [ ]:
documents

In [ ]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as accurately as possible
based on th einstructions and context provided.
"""

In [ ]:
#default format supported by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
!huggingface-cli login

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16,"load_in_8bit":True}
)


In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [ ]:
from llama_index.core import ServiceContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings


In [ ]:
embed_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
embed_model

In [ ]:
service_context=ServiceContext.from_defaults(
chunk_size=1024,
llm=llm,
embed_model=embed_model)

In [ ]:
service_context

In [ ]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [ ]:
query_engine=index.as_query_engine()
response=query_engine.query("What is attention is all you need?")
print(response)